In [1]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from imutils import build_montages
from imutils import paths
import random
import cv2

In [3]:
imagePaths = list(paths.list_images("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\train"))
random.shuffle(imagePaths)
imagePaths = imagePaths[:21]

In [4]:
# initialize the list of images
images = []
# loop over the list of image paths
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    images.append(image)
# load the image and update the list of images
# construct the montages for the images
montages = build_montages(images, (128, 196), (7, 3))

In [5]:
labels = pd.read_csv("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\Training_set_covid.csv")

labels.head()
file_paths = [[fname, 'G:/PYTHON/Final year project/Final/covid_image_data/covid_image_data/train/' + fname] for fname in labels['filename']]

if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()


train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

Number of labels i.e.  3479 matches the number of filenames i.e.  3479


,filename,filepaths,label
0,Image_1.jpg,G:/PYTHON/Final year project/Final/covid_image...,1
1,Image_2.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
2,Image_3.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
3,Image_4.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
4,Image_5.jpg,G:/PYTHON/Final year project/Final/covid_image...,0


In [6]:
test_image_order = pd.read_csv("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\Testing_set_covid.csv")
test_image_order.head()

file_paths_test = [[fname, 'G:/PYTHON/Final year project/Final/covid_image_data/covid_image_data/test/' + fname] for fname in test_image_order['filename']]

test_images = pd.DataFrame(file_paths_test, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,G:/PYTHON/Final year project/Final/covid_image...
1,Image_2.jpg,G:/PYTHON/Final year project/Final/covid_image...
2,Image_3.jpg,G:/PYTHON/Final year project/Final/covid_image...
3,Image_4.jpg,G:/PYTHON/Final year project/Final/covid_image...
4,Image_5.jpg,G:/PYTHON/Final year project/Final/covid_image...


In [7]:
data = []
image_size = 224
test_pixel_data = []
import matplotlib.pyplot as plt

for i in range(len(train_data)):
    img_array = plt.imread(train_data['filepaths'][i], cv2.COLOR_BGR2RGB)

    new_img_array = cv2.resize(img_array, (image_size, image_size))
    #new_img_array = cv2.GaussianBlur(new_img_array,(5,5),0)
    data.append([new_img_array, train_data['label'][i]])


In [8]:
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.COLOR_BGR2RGB)
    if img_array is None:
        print('Wrong path:', test_images['filepaths'][i])
    new_img_array = cv2.resize(img_array, (image_size, image_size))
    #new_img_array = cv2.GaussianBlur(new_img_array,(5,5),0)
    test_pixel_data.append(new_img_array)

In [9]:
np.random.shuffle(data)

In [10]:
x = []
y = []

counter = 406

for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [11]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([3073,  406], dtype=int64))

In [12]:
pd.Series(y).value_counts()

0    3073
1     406
dtype: int64

In [13]:
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state = 42,stratify=y)

In [14]:
test_pixel_data = np.array(test_pixel_data)

test_pixel_data = test_pixel_data/255

print(X_train.shape,X_val.shape,test_pixel_data.shape)

(2783, 224, 224, 3) (696, 224, 224, 3) (870, 224, 224, 3)


In [15]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow(X_train,y_train,batch_size=8)

val_generator = val_datagen.flow(X_val,y_val,batch_size=8)

In [16]:
pd.Series(y_val).value_counts(normalize=True)

0    0.883621
1    0.116379
dtype: float64

In [17]:
print(X_train.shape,X_val.shape)

(2783, 224, 224, 3) (696, 224, 224, 3)


In [18]:
import tensorflow as tf
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from sklearn.metrics import accuracy_score,r2_score

In [19]:
# 224
model_den = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\dense_nET_co.h5")
model_res = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\resnet50.h5")
model_inc = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\inception_v3.h5")
model_eff = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\efficientnetb0.h5")
model_vg16 = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\vgg_16_1.h5")

In [20]:
model_store = {
    "DenseNet": model_den,
    "ResNet" : model_res,
    "Inception" : model_inc,
    "Efficient Net":model_eff,
    "VGG16":model_vg16
}
model_store_name = ["DenseNet","ResNet","Inception","Efficient Net","VGG16"]

In [21]:
from sklearn.metrics import confusion_matrix,classification_report

In [22]:
from tqdm import tqdm
import time

In [26]:
for ptr1 in tqdm(range(len(model_store_name)-1)):
    for ptr2 in range(ptr1+1,len(model_store_name)):
        print(model_store_name[ptr1],model_store_name[ptr2])
        print()
        model1 = model_store[model_store_name[ptr1]]
        model2 = model_store[model_store_name[ptr2]]
        store = []
        for img in X_val:
            dt = []
            dt.append(img)
            X = np.array(dt)
            X = X/255
            preds = [model1.predict(X), model2.predict(X)]
            weights = [0.5,0.5]
            weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
            if weighted_preds > 0.5:
                store.append(1)
            else:
                store.append(0)
        print(confusion_matrix(y_val,store))
        target_names = ["COVID","Normal"]
        print(classification_report(y_val,store, target_names=target_names))

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

DenseNet ResNet

[[612   3]
 [  1  80]]
              precision    recall  f1-score   support

       COVID       1.00      1.00      1.00       615
      Normal       0.96      0.99      0.98        81

    accuracy                           0.99       696
   macro avg       0.98      0.99      0.99       696
weighted avg       0.99      0.99      0.99       696

DenseNet Inception

[[612   3]
 [  0  81]]
              precision    recall  f1-score   support

       COVID       1.00      1.00      1.00       615
      Normal       0.96      1.00      0.98        81

    accuracy                           1.00       696
   macro avg       0.98      1.00      0.99       696
weighted avg       1.00      1.00      1.00       696

DenseNet Efficient Net

[[615   0]
 [ 81   0]]
              precision    recall  f1-score   support

       COVID       0.88      1.00      0.94       615
      Normal       0.00      0.00      0.00        81

    accuracy                           0.88       69

C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

[[614   1]
 [  4  77]]
              precision    recall  f1-score   support

       COVID       0.99      1.00      1.00       615
      Normal       0.99      0.95      0.97        81

    accuracy                           0.99       696
   macro avg       0.99      0.97      0.98       696
weighted avg       0.99      0.99      0.99       696

ResNet Inception

[[591  24]
 [  1  80]]
              precision    recall  f1-score   support

       COVID       1.00      0.96      0.98       615
      Normal       0.77      0.99      0.86        81

    accuracy                           0.96       696
   macro avg       0.88      0.97      0.92       696
weighted avg       0.97      0.96      0.97       696

ResNet Efficient Net

[[615   0]
 [ 81   0]]
              precision    recall  f1-score   support

       COVID       0.88      1.00      0.94       615
      Normal       0.00      0.00      0.00        81

    accuracy                           0.88       696
   macro avg       

C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

[[605  10]
 [  2  79]]
              precision    recall  f1-score   support

       COVID       1.00      0.98      0.99       615
      Normal       0.89      0.98      0.93        81

    accuracy                           0.98       696
   macro avg       0.94      0.98      0.96       696
weighted avg       0.98      0.98      0.98       696

Inception Efficient Net

[[615   0]
 [ 81   0]]
              precision    recall  f1-score   support

       COVID       0.88      1.00      0.94       615
      Normal       0.00      0.00      0.00        81

    accuracy                           0.88       696
   macro avg       0.44      0.50      0.47       696
weighted avg       0.78      0.88      0.83       696

Inception VGG16



C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

[[604  11]
 [  1  80]]
              precision    recall  f1-score   support

       COVID       1.00      0.98      0.99       615
      Normal       0.88      0.99      0.93        81

    accuracy                           0.98       696
   macro avg       0.94      0.98      0.96       696
weighted avg       0.98      0.98      0.98       696

Efficient Net VGG16



C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

[[615   0]
 [ 81   0]]
              precision    recall  f1-score   support

       COVID       0.88      1.00      0.94       615
      Normal       0.00      0.00      0.00        81

    accuracy                           0.88       696
   macro avg       0.44      0.50      0.47       696
weighted avg       0.78      0.88      0.83       696



In [25]:
for ptr1 in tqdm(range(len(model_store_name)-1)):
    for ptr2 in range(ptr1+1,len(model_store_name)):
        for ptr3 in range(ptr2+1,len(model_store_name)):
            print(model_store_name[ptr1],model_store_name[ptr2], model_store_name[ptr3])
            print()
            model1 = model_store[model_store_name[ptr1]]
            model2 = model_store[model_store_name[ptr2]]
            model3 = model_store[model_store_name[ptr3]]
            store = []
            for img in X_val:
                dt = []
                dt.append(img)
                X = np.array(dt)
                X = X/255
                preds = [model1.predict(X), model2.predict(X), model3.predict(X)]
                weights = [0.33,0.33,0.33]
                weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
                if weighted_preds > 0.5:
                    store.append(1)
                else:
                    store.append(0)
            print(confusion_matrix(y_val,store))
            target_names = ["COVID","Normal"]
            print(classification_report(y_val,store, target_names=target_names))

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

DenseNet ResNet Inception

[[608   7]
 [  0  81]]
              precision    recall  f1-score   support

       COVID       1.00      0.99      0.99       615
      Normal       0.92      1.00      0.96        81

    accuracy                           0.99       696
   macro avg       0.96      0.99      0.98       696
weighted avg       0.99      0.99      0.99       696

DenseNet ResNet Efficient Net

[[613   2]
 [  3  78]]
              precision    recall  f1-score   support

       COVID       1.00      1.00      1.00       615
      Normal       0.97      0.96      0.97        81

    accuracy                           0.99       696
   macro avg       0.99      0.98      0.98       696
weighted avg       0.99      0.99      0.99       696

DenseNet ResNet VGG16

[[611   4]
 [  2  79]]
              precision    recall  f1-score   support

       COVID       1.00      0.99      1.00       615
      Normal       0.95      0.98      0.96        81

    accuracy                    

 25%|████████████████████▌                                                             | 1/4 [16:56<50:50, 1016.98s/it]

[[615   0]
 [ 38  43]]
              precision    recall  f1-score   support

       COVID       0.94      1.00      0.97       615
      Normal       1.00      0.53      0.69        81

    accuracy                           0.95       696
   macro avg       0.97      0.77      0.83       696
weighted avg       0.95      0.95      0.94       696

ResNet Inception Efficient Net

[[609   6]
 [  3  78]]
              precision    recall  f1-score   support

       COVID       1.00      0.99      0.99       615
      Normal       0.93      0.96      0.95        81

    accuracy                           0.99       696
   macro avg       0.96      0.98      0.97       696
weighted avg       0.99      0.99      0.99       696

ResNet Inception VGG16

[[608   7]
 [  3  78]]
              precision    recall  f1-score   support

       COVID       1.00      0.99      0.99       615
      Normal       0.92      0.96      0.94        81

    accuracy                           0.99       696
   

 50%|█████████████████████████████████████████▌                                         | 2/4 [23:44<21:57, 658.63s/it]

[[614   1]
 [ 38  43]]
              precision    recall  f1-score   support

       COVID       0.94      1.00      0.97       615
      Normal       0.98      0.53      0.69        81

    accuracy                           0.94       696
   macro avg       0.96      0.76      0.83       696
weighted avg       0.95      0.94      0.94       696

Inception Efficient Net VGG16



100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [25:57<00:00, 389.40s/it]

[[614   1]
 [ 38  43]]
              precision    recall  f1-score   support

       COVID       0.94      1.00      0.97       615
      Normal       0.98      0.53      0.69        81

    accuracy                           0.94       696
   macro avg       0.96      0.76      0.83       696
weighted avg       0.95      0.94      0.94       696

